In [8]:
import urllib.request

# ------------------------
# 1. Define URLs in Python
# ------------------------
PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL = f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{PREFIX}/hair_classifier_v1.onnx"

print("Data URL:", DATA_URL)
print("Model URL:", MODEL_URL)

# ------------------------
# 2. Download function
# ------------------------
def download(url, filename=None):
    if filename is None:
        filename = url.split("/")[-1]
    print(f"⬇️  Downloading {filename} ...")
    urllib.request.urlretrieve(url, filename)
    print(f"✅ Saved as {filename}\n")

# ------------------------
# 3. Download both files
# ------------------------
download(DATA_URL)
download(MODEL_URL)


Data URL: https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Model URL: https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx
⬇️  Downloading hair_classifier_v1.onnx.data ...
✅ Saved as hair_classifier_v1.onnx.data

⬇️  Downloading hair_classifier_v1.onnx ...
✅ Saved as hair_classifier_v1.onnx



In [14]:
import onnx

# Load your model
model = onnx.load("hair_classifier_v1.onnx")

# The ONNX graph
graph = model.graph

print("=== INPUTS ===")
for inp in graph.input:
    print(inp.name)

print("\n=== OUTPUTS ===")
for out in graph.output:
    print(out.name)

print("\n=== ALL NODES (for debugging) ===")
for n in graph.node:
    print(n.op_type, "->", n.output)


=== INPUTS ===
input

=== OUTPUTS ===
output

=== ALL NODES (for debugging) ===
Shape -> ['val_0']
Conv -> ['conv2d']
Relu -> ['relu']
MaxPool -> ['max_pool2d']
Concat -> ['val_5']
Reshape -> ['view']
Gemm -> ['linear']
Relu -> ['relu_1']
Gemm -> ['output']


In [15]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [16]:
!wget https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

--2025-12-08 22:53:49--  https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg
Resolving habrastorage.org (habrastorage.org)... 2a14:b680:0:56::35, 2a14:b680:0:56::34, 95.47.173.34, ...
Connecting to habrastorage.org (habrastorage.org)|2a14:b680:0:56::35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 398272 (389K) [image/jpeg]
Saving to: 'yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

     0K .......... .......... .......... .......... .......... 12% 1.59M 0s
    50K .......... .......... .......... .......... .......... 25% 1014K 0s
   100K .......... .......... .......... .......... .......... 38% 1.10M 0s
   150K .......... .......... .......... .......... .......... 51% 1.02M 0s
   200K .......... .......... .......... .......... .......... 64% 21.5M 0s
   250K .......... .......... .......... .......... .......... 77% 17.3M 0s
   300K .......... .......... .......... .......... .......... 89% 1.05M 0s
   350K .......... .......... .......... ......

In [17]:
import onnxruntime as ort
session = ort.InferenceSession("hair_classifier_v1.onnx")
session.get_inputs()[0].shape


['s77', 3, 200, 200]

In [22]:
from PIL import Image
import torchvision.transforms as transforms
import torch

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

img = Image.open("yf_dokzqy3vcritme8ggnzqlvwa.jpeg").convert("RGB")
x = train_transforms(img)

# Print first pixel (R channel, row 0, col 0)
print("First pixel R channel:", x[0, 0, 0])


First pixel R channel: tensor(-1.0562)


In [23]:
import onnxruntime as ort
import torchvision.transforms as transforms
from PIL import Image
import torch
import numpy as np

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


img = Image.open("yf_dokzqy3vcritme8ggnzqlvwa.jpeg").convert("RGB")
x = train_transforms(img)              
x = x.unsqueeze(0).numpy()             


session = ort.InferenceSession("hair_classifier_v1.onnx")

# Get input name
input_name = session.get_inputs()[0].name

# -------------------------------
# 4. Run inference
# -------------------------------
outputs = session.run(None, {input_name: x})
logits = outputs[0][0]   # extract first batch

print("Raw model output (logits):", logits)

# -------------------------------
# 5. Convert logits → probability
#    SIGMOID (binary classification)
# -------------------------------
proba = 1 / (1 + np.exp(-logits))
print("Probability:", proba)


Raw model output (logits): [0.72769666]
Probability: [0.6742996]


In [25]:
!docker pull agrigorev/model-2025-hairstyle:v1


v1: Pulling from agrigorev/model-2025-hairstyle
cd9c39ee4ab8: Pulling fs layer
d2379533db7f: Pulling fs layer
b71754c34aa3: Pulling fs layer
94f2ecca3b37: Pulling fs layer
15b946295de2: Pulling fs layer
26c6d8a1e1c2: Pulling fs layer
f9e394d707b7: Pulling fs layer
55d9a27bb275: Pulling fs layer
cd9c39ee4ab8: Download complete
d2379533db7f: Download complete
15b946295de2: Download complete
26c6d8a1e1c2: Download complete
94f2ecca3b37: Download complete
b71754c34aa3: Download complete
b71754c34aa3: Pull complete
d2379533db7f: Pull complete
26c6d8a1e1c2: Pull complete
94f2ecca3b37: Pull complete
55d9a27bb275: Download complete
f9e394d707b7: Download complete
f9e394d707b7: Pull complete
15b946295de2: Pull complete
55d9a27bb275: Pull complete
cd9c39ee4ab8: Pull complete
Digest: sha256:9e43d5a5323f7f07688c0765d3c0137af66d0154af37833ed721d6b4de6df528
Status: Downloaded newer image for agrigorev/model-2025-hairstyle:v1
docker.io/agrigorev/model-2025-hairstyle:v1


In [26]:
!docker images


REPOSITORY                       TAG       IMAGE ID       CREATED        SIZE
agrigorev/model-2025-hairstyle   v1        9e43d5a5323f   7 days ago     921MB
zoomcamp-hw5                     latest    b4394407c4b1   5 weeks ago    216MB
agrigorev/zoomcamp-model         2025      14d79fde0bbf   6 weeks ago    181MB
hello-world                      latest    56433a6be3fd   4 months ago   20.3kB


In [ ]:
...

'Invoke-WebRequest' is not recognized as an internal or external command,
operable program or batch file.
